In [1]:
import torch 
from torch.autograd import Variable 
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
corpus = [
    'he is a king',
    'she is a queen',
    'she is a woman',
    'he is a man',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',
]

## split sentence into words

In [3]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens
tokenized_corpus = tokenize_corpus(corpus)

## construct vocabulary

In [5]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2ind = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

## sample (center_word, context_word) from corpus

In [8]:
window_size = 2
idx_pairs = []
for sentence in tokenized_corpus:
    for i in range(len(sentence)):
        center_word = sentence[i]
        left = max(0, i-window_size)
        right = min(len(sentence) - 1, i+window_size)
        for j in range(left, right):
            if j != i:
                context_word = sentence[j]
                idx_pairs.append((word2ind[center_word], word2ind[context_word]))

idx_pairs = np.array(idx_pairs)

## Model & Target & Algorithm
值得注意的是，我们在这次并没有为model独立设计一个类，所以需要在for epoch循环中，描述forward的过程，原范式如下：
```
model {
    __init__()
    forward()
}

m = model()

for epo in num_epochs:
    y_pred = m(x)
    ...
```
新范式如下：
```
__init__() # parameter needs to be trained goes here.

for epo in num_epochs:
    y_pred = forward()
    ...
```

In [1]:
def get_input_layer(word_idx):
    """
        construct input tensor from data
        torch的模型输入必须为tensor，所以在这里将word_idx转化为tensor
    """
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [2]:
embedding_dims = 5

# model __init__
W1 = torch.randn(embedding_dims, vocabulary_size, requires_grad=True).float() 
W2 = torch.randn(vocabulary_size, embedding_dims, requires_grad=True).float()

# hyperparameter
num_epochs = 101
lr = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = get_input_layer(data).float()
        y_true = torch.from_numpy(np.array([target]))
        
        # forward: z2 = model(x) <=> z2 = W2 * W1 * x
        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
        
        # compute loss
        log_softmax = F.log_softmax(z2, dim=0)
        loss = F.nll_loss(log_softmax.view(1, -1), y_true)
        loss_val += loss.item()
        
        # backpropagation
        loss.backward()
        
        # update parameter
        W1.data -= lr * W1.grad.data
        W2.data -= lr * W2.grad.data
        
        # zero grads
        W1.grad.data.zero_()
        W1.grad.data.zero_()
        
    if epo % 10 == 0:
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

NameError: name 'vocabulary_size' is not defined